In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import DatasetDict, Dataset, load_from_disk
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder

In [2]:
path_to_retrieve = "../tokenized_dataset"


In [3]:

dataset_dict = load_from_disk(path_to_retrieve)

In [4]:
set(dataset_dict['train']['labels'])

{0, 1, 2, 3, 4}

In [5]:
model_name = "bert-base-uncased"

#model_name = 'bert-large-uncased'

In [6]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def compute_metrics(p):
    logits, labels = p.predictions, p.label_ids
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [8]:
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='/results',
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_steps=10,
    save_total_limit=2,
    remove_unused_columns=False,
    run_name='run_name',
    logging_dir='/logs',
    logging_steps=10,
    load_best_model_at_end=True,
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    compute_metrics=compute_metrics, 
)

In [10]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy
10,1.484300,1.263397,0.655000
20,0.997600,0.811025,0.850000
30,0.669900,0.521227,0.915000
40,0.459000,0.278449,0.980000
50,0.256800,0.143181,1.000000
60,0.160100,0.093543,1.000000
70,0.096400,0.057435,1.000000
80,0.062200,0.042969,1.000000
90,0.049100,0.036593,1.000000
100,0.045700,0.034267,1.000000


TrainOutput(global_step=100, training_loss=0.42811209440231324, metrics={'train_runtime': 56.1001, 'train_samples_per_second': 14.26, 'train_steps_per_second': 1.783, 'total_flos': 210494513971200.0, 'train_loss': 0.42811209440231324, 'epoch': 1.0})